Fire Emblem 4 Character Predictor
============================

In [10]:
from __future__ import print_function
%matplotlib inline
import numpy as np
from scipy.stats import binom, norm
from matplotlib import pyplot as plt
import pandas as pd
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
# from ipywidgets.embed import embed_minimal_html
# from IPython.display import Javascript

In [11]:
test = pd.ExcelFile('FE4.xlsx')

In [15]:
# function for name dropdown
def sel(Name):
    df = pd.read_excel(test, Name)

    ln = len(df)-1

    # starting level
    slv = df.iloc[ln][3]

    # build dict of growths
    growths = {df.iloc[i][0] : df.iloc[i][3] for i in range(0, int(ln/2))}

    # build dict of bases
    bases = {df.iloc[i][0] : df.iloc[i][3] for i in range(int(ln/2), ln)}
    
    # select stat
    def f(Stat):
        # toggle level
        def g(Level=slv+1):
            # distribution stuff

            n = Level-slv
            p = list(growths.values())[Stat]

            bs = list(bases.values())[Stat]

            if p > 1:
                p = p-1
                bs+=n

            # binomial dist of growth for given stat
            bn = binom(n, p)

            x = np.arange(binom.ppf(0.01, n, p), binom.ppf(0.99, n, p))

            fig = plt.figure()
            # plt.scatter(x, binom.pmf(x, n, p))
            plt.bar(x+bs, binom.pmf(x, n, p))
            plt.xlabel('Stat Value')
            plt.ylabel('Probability')
            plt.title(f' {Name}: Expected {list(bases.keys())[Stat]} Value at Level {int(n+slv)}')

            # y = np.linspace(norm.ppf(0.01), norm.ppf(0.99), 100)
            y = np.linspace(binom.ppf(0.01, n, p), binom.ppf(0.99, n, p), 100)
            # y = (x - binom.mean(n, p))/binom.std(n, p)

            # plt.hold(True)
            plt.plot(y+bs, norm.pdf(y, loc=binom.mean(n, p), scale=binom.std(n, p)), '-r')
            plt.show()

            print(f'At Level {int(n+slv)}, Expected {list(bases.keys())[Stat]} = {int(np.floor(binom.mean(n, p))+bs)}')
            print(f'With Standard Deviation = {(int(np.floor(binom.std(n, p))))}')

            print(f'{Name}: Average Stats at Level {int(n+slv)}:')
            for i, v in enumerate(list(bases.keys())):
                p = list(growths.values())[i]
                bs = list(bases.values())[i]
                
                if p > 1:
                    p = p-1
                    bs+=n

                print(f'Average {v} = {int(np.floor(binom.mean(n, p)+bs))}')

            return Level

        lv = interactive(g, Level=(int(slv+1), 30, 1), continuous_update=True)
        display(lv)
        return Stat

    s = interactive(f, Stat=[('HP', 0), ('STR', 1), ('MAG', 2), ('SKL', 3),
                            ('SPD', 4), ('DEF', 5), ('RES', 6), ('LCK', 7)], continuous_update=True)
    display(s)
    
    return Name

# name dropdown
t = interactive(sel, Name=test.sheet_names, continuous_update=True)
display(t) # show

interactive(children=(Dropdown(description='Name', options=('Alec', 'Arden', 'Ayra', 'Azel', 'Beowulf', 'Brigi…